In [1]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas

In [2]:
# load dataset
diabetes = datasets.load_diabetes(as_frame=True)
X = diabetes.data.values
y = diabetes.target.values
X.shape

# train test spliting
test_size=0.2
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42)

# Standardize features
scaler=StandardScaler()
Xtr= scaler.fit_transform(Xtr)
Xte= scaler.transform(Xte)

In [3]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size=1, dropout_prob=0.5):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.out = nn.Linear(64, output_size)
        
        self.dropout = nn.Dropout(p=dropout_prob)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.out(x)
        return x

In [4]:
num_epochs=100
lr=0.0005
dropout=0.1
batch_size=64

In [5]:
Xtr = torch.tensor(Xtr, dtype=torch.float32)
ytr = torch.tensor(ytr, dtype=torch.float32)
Xte = torch.tensor(Xte, dtype=torch.float32)
yte = torch.tensor(yte, dtype=torch.float32)

# Wrap Xtr and ytr into a dataset
train_dataset = TensorDataset(Xtr, ytr)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [6]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_size=Xtr.shape[1], dropout_prob=dropout).to(device)
criterion = nn.BCEWithLogitsLoss()  # for binary classification
criterion = nn.MSELoss() #for regression
optimizer = optim.Adam(model.parameters(), lr=lr)

In [7]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch_x, batch_y in train_dataloader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        logits = model(batch_x)
        loss = criterion(logits, batch_y.view(-1, 1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/100], Loss: 30310.5420
Epoch [2/100], Loss: 29243.5394
Epoch [3/100], Loss: 30044.5247
Epoch [4/100], Loss: 29860.4508
Epoch [5/100], Loss: 29696.7614
Epoch [6/100], Loss: 29713.8717
Epoch [7/100], Loss: 30467.9827
Epoch [8/100], Loss: 29906.8190
Epoch [9/100], Loss: 28805.7611
Epoch [10/100], Loss: 28257.5947
Epoch [11/100], Loss: 27452.0605
Epoch [12/100], Loss: 26421.7109
Epoch [13/100], Loss: 25010.4076
Epoch [14/100], Loss: 22369.2793
Epoch [15/100], Loss: 18765.8636
Epoch [16/100], Loss: 15069.2697
Epoch [17/100], Loss: 10533.2059
Epoch [18/100], Loss: 6867.5565
Epoch [19/100], Loss: 5285.3045
Epoch [20/100], Loss: 5295.5701
Epoch [21/100], Loss: 5268.2492
Epoch [22/100], Loss: 4695.7739
Epoch [23/100], Loss: 4467.4322
Epoch [24/100], Loss: 4385.3983
Epoch [25/100], Loss: 4145.1329
Epoch [26/100], Loss: 3992.7265
Epoch [27/100], Loss: 4248.1073
Epoch [28/100], Loss: 3983.7410
Epoch [29/100], Loss: 3795.8305
Epoch [30/100], Loss: 3942.8896
Epoch [31/100], Loss: 3736.4460


In [8]:
y_pred=model(Xte)
#print(f'ACC:{accuracy_score(yte.detach().numpy(),y_pred.detach().numpy()>0.5)}') #classification
print(f'MSE:{mean_squared_error(yte.detach().numpy(),y_pred.detach().numpy())}') #regression

MSE:3155.85986328125
